In [11]:
import pandas as pd

# 加载数据文件
data_path = "jq_stock_ticks.csv"
data = pd.read_csv(data_path)

# 初始化变量
capital = 100000  # 初始资本
shares = 0  # 持有股票数量
buy_sell_log = []  # 存储买入和卖出的日志
trade_count = 0  # 交易次数
support_counter = {price: 0 for price in data[['b1_p', 'b2_p', 'b3_p', 'b4_p', 'b5_p']].values.flatten()}
resistance_counter = {price: 0 for price in data[['a1_p', 'a2_p', 'a3_p', 'a4_p', 'a5_p']].values.flatten()}
support_level = None
resistance_level = None
prev_date = None

# 遍历每一条数据
for idx, row in data.iterrows():
    timestamp = row['time']
    current_date = pd.to_datetime(timestamp).date()
    current_price = row['current']

    if current_date != prev_date: # 新的交易日
        # 重置支撑位和压力位计数器
        support_counter = {price: 0 for price in support_counter.keys()}
        resistance_counter = {price: 0 for price in resistance_counter.keys()}
        support_level = None
        resistance_level = None

    # 累计挂买盘和挂卖盘
    for i in range(1, 6):
        support_counter[row[f'b{i}_p']] += row[f'b{i}_v']
        resistance_counter[row[f'a{i}_p']] += row[f'a{i}_v']

    # 计算支撑位和压力位
    support_level = max(support_counter, key=support_counter.get)
    resistance_level = max(resistance_counter, key=resistance_counter.get)

    # 判断买卖条件
    if current_price <= support_level and capital > 0:  # 买入条件
        purchase_shares = capital / current_price * (1 - 0.0003)  # 扣除交易费
        shares += purchase_shares
        capital = 0
        buy_sell_log.append((timestamp, 'buy', current_price))
        trade_count += 1
    elif current_price >= resistance_level and shares > 0:  # 卖出条件
        capital += shares * current_price * (1 - 0.0003)  # 扣除交易费
        shares = 0
        buy_sell_log.append((timestamp, 'sell', current_price))
        trade_count += 1

    prev_date = current_date

# 清仓（如果还持有股票）
if shares > 0:
    capital += shares * data['current'].iloc[-1] * (1 - 0.0003)
    timestamp = data['time'].iloc[-1]
    buy_sell_log.append((timestamp, 'sell', data['current'].iloc[-1]))

# 打印买入和卖出日志
for log in buy_sell_log:
    print(f"Time: {log[0]}, Action: {log[1]}, Price: {log[2]:.2f}")

initial_capital = 100000
final_capital = capital
profit_percentage = (final_capital - initial_capital) / initial_capital * 100

print(f"最终资本: {final_capital:.2f}")
print(f"收益率: {profit_percentage:.2f}%")
print(f"总交易次数: {trade_count}")


Time: 2022-10-10 10:34:06, Action: buy, Price: 2.10
Time: 2022-10-11 09:38:12, Action: sell, Price: 2.06
Time: 2022-10-11 09:40:00, Action: buy, Price: 2.04
Time: 2022-10-11 10:41:00, Action: sell, Price: 2.05
Time: 2022-10-12 10:13:00, Action: buy, Price: 2.02
Time: 2022-10-12 13:45:12, Action: sell, Price: 2.06
Time: 2022-10-13 09:34:18, Action: buy, Price: 2.07
Time: 2022-10-14 09:32:45, Action: sell, Price: 2.10
Time: 2022-10-14 09:58:12, Action: buy, Price: 2.11
Time: 2022-10-14 11:13:21, Action: sell, Price: 2.14
Time: 2022-10-14 14:00:51, Action: buy, Price: 2.10
Time: 2022-10-17 09:56:15, Action: sell, Price: 2.10
Time: 2022-10-18 13:52:24, Action: buy, Price: 2.13
Time: 2022-10-20 10:34:39, Action: sell, Price: 2.09
Time: 2022-10-21 09:52:03, Action: buy, Price: 2.08
Time: 2022-10-24 09:35:36, Action: sell, Price: 2.04
Time: 2022-10-24 09:58:57, Action: buy, Price: 2.01
Time: 2022-10-25 11:19:57, Action: sell, Price: 1.93
Time: 2022-10-26 09:39:30, Action: buy, Price: 1.93
Tim